In [31]:
import pandas
#https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DEMO.htm
#https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_BMX.htm
#https://data.census.gov/table/ACSST1Y2022.S2001?q=American%20Community%20Survey%20Income%20by%20State%20and%20County

In [32]:
# Retrieve data sets
demodf_init = pandas.read_sas('data/P_DEMO.XPT')
measuredf_init = pandas.read_sas('data/P_BMX.XPT')
censusdf_init = pandas.read_csv('data/census.csv')
coldf_init = pandas.read_csv('data/column.csv')

In [33]:
# Format CDC data with relevant columns
dvars = ["RIAGENDR", "RIDAGEYR", "DMDMARTZ", "DMDEDUC2"]
demodf = demodf_init[dvars].copy()
mvars = ["BMXHT", "BMXWT"]
measuredf = measuredf_init[mvars].copy()

In [34]:
# Rename incomprehensible column names
mdf = measuredf.rename(columns={"BMXHT" : "Height(cm)", "BMXWT" : "Weight(kg)"}).copy()
ddf = demodf.rename(columns={"RIAGENDR" : "Gender", "RIDAGEYR" : "Age", 
                      "DMDMARTZ" : "Relationship", "DMDEDUC2": "Education"}).copy()
df = pandas.concat([ddf, mdf]).fillna(0)
df.head()

,Gender,Age,Relationship,Education,Height(cm),Weight(kg)
0,1.0,2.0,0.0,0.0,0.0,0.0
1,2.0,13.0,0.0,0.0,0.0,0.0
2,1.0,2.0,0.0,0.0,0.0,0.0
3,2.0,29.0,3.0,5.0,0.0,0.0
4,2.0,21.0,3.0,4.0,0.0,0.0


In [35]:
# df.to_csv('cdc.csv', index=False)

In [36]:
cols = censusdf_init.columns
censusdf_init.columns

Index(['Label (Grouping)', 'Alabama!!Total!!Estimate',
       'Alabama!!Total!!Margin of Error', 'Alabama!!Percent!!Estimate',
       'Alabama!!Percent!!Margin of Error', 'Alabama!!Male!!Estimate',
       'Alabama!!Male!!Margin of Error', 'Alabama!!Percent Male!!Estimate',
       'Alabama!!Percent Male!!Margin of Error', 'Alabama!!Female!!Estimate',
       ...
       'Trujillo Alto Municipio, Puerto Rico!!Percent!!Estimate',
       'Trujillo Alto Municipio, Puerto Rico!!Percent!!Margin of Error',
       'Trujillo Alto Municipio, Puerto Rico!!Male!!Estimate',
       'Trujillo Alto Municipio, Puerto Rico!!Male!!Margin of Error',
       'Trujillo Alto Municipio, Puerto Rico!!Percent Male!!Estimate',
       'Trujillo Alto Municipio, Puerto Rico!!Percent Male!!Margin of Error',
       'Trujillo Alto Municipio, Puerto Rico!!Female!!Estimate',
       'Trujillo Alto Municipio, Puerto Rico!!Female!!Margin of Error',
       'Trujillo Alto Municipio, Puerto Rico!!Percent Female!!Estimate',
      

In [77]:
# Gets rid of columns I do not care about
old_cols = [col for col in cols if "Error" not in col and "Puerto" not in col and
            "Percent" not in col and "County" not in col and "," not in col]
# Formats columns into more readable text
def removeEst(data):
    col_lst = []
    for title in data:
        if "Estimate" in title:
            col_lst.append(("_").join(title.split("!!")[:-1]))
        else:
            col_lst.append(title)
    return col_lst
new_cols = removeEst(old_cols.copy())

In [78]:
# Renames columns to be more readable
def createMapping(old_cols, new_cols):
    mapping = {}
    for old, new in zip(old_cols, new_cols):
        mapping[old] = new
    return mapping
rename_cols = createMapping(old_cols, new_cols)

In [96]:
cens_renamed_df = censusdf_init[old_cols].rename(columns=rename_cols).fillna('0').set_index('Label (Grouping)')

for col in cens_renamed_df.columns:
    cens_renamed_df[col] = cens_renamed_df[col].str.replace(',', '').astype(int)

cens_renamed_df["Total"] = cens_renamed_df.sum(axis=1)
cens_renamed_df.head(21)

,Alabama_Total,Alabama_Male,Alabama_Female,Alaska_Total,Alaska_Male,Alaska_Female,Arizona_Total,Arizona_Male,Arizona_Female,Arkansas_Total,...,West Virginia_Total,West Virginia_Male,West Virginia_Female,Wisconsin_Total,Wisconsin_Male,Wisconsin_Female,Wyoming_Total,Wyoming_Male,Wyoming_Female,Total
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Population 16 years and over with earnings,2516192,1308524,1207668,407800,230522,177278,3753211,2023859,1729352,1458408,...,810082,426516,383566,3308913,1742868,1566045,323741,174588,149153,356059956
Median earnings (dollars),36953,44443,30824,47357,53127,41567,42346,47810,37718,36761,...,37256,44476,31364,42873,50488,36604,39954,49839,32207,6636543
"FULL-TIME, YEAR-ROUND WORKERS WITH EARNINGS",1723646,964406,759240,255724,152785,102939,2559331,1478889,1080442,1012797,...,548802,306208,242594,2178536,1241897,936639,212802,126510,86292,238332600
"$1 to $9,999 or loss",38226,16913,21313,3189,2161,1028,43204,22737,20467,17693,...,10741,6093,4648,33317,18571,14746,4278,2144,2134,3872398
"$10,000 to $14,999",50954,21730,29224,4024,2498,1526,48574,24377,24197,24611,...,11228,5480,5748,33097,17852,15245,4208,2069,2139,4580886
"$15,000 to $24,999",175399,73921,101478,12662,7048,5614,141666,71471,70195,92565,...,53310,22293,31017,108769,47133,61636,13379,7467,5912,15573998
"$25,000 to $34,999",266047,124313,141734,29440,16218,13222,332808,171777,161031,178108,...,83668,37786,45882,245448,105751,139697,27557,13936,13621,29422224
"$35,000 to $49,999",337591,172012,165579,42610,22373,20237,558630,296489,262141,239553,...,117635,56983,60652,481048,239222,241826,41671,20785,20886,45455032
"$50,000 to $64,999",285446,160850,124596,44456,25261,19195,463601,271231,192370,176763,...,97978,60192,37786,437147,254618,182529,42548,25073,17475,39255882


In [ ]:
def findAmout(income):
    if income >= 100000:
        return 11
    elif income >= 75000:
        return 10
    elif income >= 65000:
        return 9
    elif income >= 50000:
        return 8
    elif income >= 35000:
        return 7
    elif income >= 25000:
        return 6
    elif income >= 15000:
        return 5
    elif income >= 10000:
        return 4
    else return 3

In [82]:
# Checking if we have correct amount of states
states = []
for col in cens_renamed_df.columns:
    state = col.split("_")[0]
    if state not in states:
        states.append(state)
len(states[1:])

51